# Train a online portfolio

In [86]:
import jax.numpy as jnp
import jax
import optax
import polars as pl
import polars.selectors as cs

from pyfoliopt.loss import neg_logdot_loss

In [ ]:
import pandas as pd
import yfinance as yf

data: pd.DataFrame = yf.download("BTC-USD ETH-USD", start="2018-01-01", end="2019-01-01")["Close"].dropna()

In [ ]:
data.plot(grid=True, subplots=True)

In [ ]:
df_prices = pl.from_dataframe(data.reset_index()) 
df_prices

In [ ]:
df_returns = (
    df_prices
    .select(
        cs.datetime(),
        pl.col("BTC-USD").pct_change().alias("BTC-USD_long"),
        pl.col("ETH-USD").pct_change().mul(-1).alias("ETH-USD_short")
    )
)
df_returns

In [ ]:
(
    df_returns
    .with_columns(
        CRP = pl.col("BTC-USD_long") * 0.5 + pl.col("ETH-USD_short") * 0.5,
    )
    .select(cs.datetime(), cs.numeric().add(1).cum_prod())
    .to_pandas()
    .set_index("Date")
    .plot(grid=True)
)